In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('cleaned_tweets_with_labels_v_b64.csv')
df.head()

,tweet_id,clean_tweet,user_id,lat,long,city,country,state,state_code,political_label,label_score
0,1.316529e+18,elecciones2020 en florida joebiden dice que do...,3.606665e+08,25.774270,-80.193660,NaN,United States of America,Florida,FL,Non-Political,0.142214
1,1.316529e+18,trump as a student i used to hear for years fo...,8.436472e+06,45.520247,-122.674195,Portland,United States of America,Oregon,OR,Non-Political,0.194428
2,1.316529e+18,you get a tie and you get a tie trump s rally ...,4.741380e+07,38.894992,-77.036558,Washington,United States of America,District of Columbia,DC,Non-Political,0.085516
3,1.316529e+18,her 15 minutes were over long time ago omarosa...,1.138416e+09,33.782519,-117.228648,NaN,United States of America,California,CA,Non-Political,0.258789
4,1.316529e+18,there wont be many of them unless you all have...,9.007611e+17,40.225357,-82.688140,NaN,United States of America,Ohio,OH,Voting Rights,0.474250


In [6]:
df = df[df['country']=='United States of America']
print(df.shape)

(301340, 11)


In [7]:
df = df[df['political_label'] != 'Non-Political']

In [8]:
df.to_csv('cleaned_tweets_with_labels_v_b64_usa.csv', index=False)
df = pd.read_csv('cleaned_tweets_with_labels_v_b64_usa.csv')
print(df.shape)

(44258, 11)


In [9]:
!pip install transformers torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [10]:
from transformers import pipeline
from tqdm import tqdm

In [11]:
df2 = df['political_label'].unique()
print(df2)

['Voting Rights' 'Taxes' 'Free Speech' 'Healthcare' 'Energy Policy'
 'Foreign Policy' 'Criminal Justice' 'Police Reform' 'Inflation'
 'Abortion' 'Immigration' 'Climate Change' 'Gun Control' 'Education'
 'Unemployment' 'Social Security' 'LGBTQ Rights' 'Welfare' 'Minimum Wage'
 'Student Loans']


In [12]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [136]:
df_student_loans = df[df['political_label']=='Student Loans'].copy()
labels = [
    "the author supports student loan forgiveness or debt relief",
    "the author opposes student loan forgiveness or debt relief",
    "the author does not express a clear stance on student loans"
]

hypothesis_template = "The tweet implies that {}."
stances = []

In [137]:
for tweet in tqdm(df_student_loans['clean_tweet'], desc="Classifying Stance"):
    result = classifier(tweet, candidate_labels=labels, hypothesis_template=hypothesis_template)
    stances.append(result['labels'][0])

df_student_loans['stance'] = stances

Classifying Stance: 100%|██████████| 86/86 [00:06<00:00, 12.54it/s]


In [138]:
df_student_loans.to_csv("cleaned_tweets_with_labels_v_b64_usa_student_loans.csv", index=False)

In [139]:
df_student_loans= pd.read_csv("cleaned_tweets_with_labels_v_b64_usa_student_loans.csv")
print(df_student_loans[['stance','clean_tweet']])

                                               stance  \
0   the author does not express a clear stance on ...   
1   the author does not express a clear stance on ...   
2   the author does not express a clear stance on ...   
3   the author does not express a clear stance on ...   
4   the author does not express a clear stance on ...   
..                                                ...   
81  the author does not express a clear stance on ...   
82  the author does not express a clear stance on ...   
83  the author does not express a clear stance on ...   
84  the author does not express a clear stance on ...   
85  the author supports student loan forgiveness o...   

                                          clean_tweet  
0   oh this may shock you but the trump administra...  
1                           trump is how much in debt  
2   trumps plan would lead to a bifurcated market ...  
3                   trump more than 1 billion in debt  
4                   trump more than

In [140]:
df_student_loans['stance'].value_counts()

,count
stance,
the author does not express a clear stance on student loans,51
the author supports student loan forgiveness or debt relief,30
the author opposes student loan forgiveness or debt relief,5
